In [19]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

/home/ubuntu/varios/skforecast


In [29]:
# Libraries
# ==============================================================================
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
import skforecast
import sklearn
import numpy as np
import pandas as pd
print(f"skforecast: {skforecast.__version__}")
print(f"sklearn: {sklearn.__version__}")
print(f"numpy: {np.__version__}")
print(f"pandas: {pd.__version__}")

skforecast: 0.13.0
sklearn: 1.5.1
numpy: 1.26.4
pandas: 2.2.2


In [21]:
# Loading the data
# ==============================================================================
df_test = pd.read_excel(
    io          = "df_test.xlsx",
    sheet_name  = 0,
    index_col   = 0,
    header      = 0,
    parse_dates = ["ds"],
)

df_test = df_test.asfreq("D")
df_test

,t_cv_01640000,t_cv_54790000,y,t_cv_01640000_soma_3_dias,t_cv_54790000_soma_3_dias,t_cv_01640000_soma_7_dias,t_cv_54790000_soma_7_dias,t_cv_01640000_media_30_dias,t_cv_54790000_media_30_dias,month,season
ds,,,,,,,,,,,
2013-01-30,0.0,0.142857,158.487202,0.6,6.135714,8.2,9.825714,5.566667,4.645886,1,1
2013-01-31,0.0,0.142857,158.487202,0.0,0.428571,8.0,8.161429,5.546667,4.455648,1,1
2013-02-01,0.0,0.142857,158.487202,0.0,0.428571,5.8,7.404286,5.546667,4.455648,2,1
2013-02-02,0.0,0.142857,158.487202,0.0,0.428571,5.8,7.404286,5.106667,4.460410,2,1
2013-02-03,0.0,0.142857,158.487202,0.0,0.428571,0.6,6.707143,5.106667,4.460410,2,1
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,0.0,0.000000,211.450000,0.0,0.000000,12.1,4.000000,2.883333,2.900000,12,1
2023-12-28,0.0,0.000000,167.420833,0.0,0.000000,0.0,0.000000,2.883333,2.900000,12,1
2023-12-29,0.0,0.600000,117.235417,0.0,0.600000,0.0,0.600000,2.883333,2.920000,12,1


In [42]:
# Forecast horizon
# ==============================================================================
f = 1 # a day ahead

# Creating the Forecaster
# ==============================================================================
transformer_y    = FunctionTransformer(func=np.log1p, inverse_func=np.expm1, validate=True)
transformer_y    = StandardScaler()

forecaster = ForecasterAutoreg(
    regressor        = LinearRegression(),
    lags             = 3,
    transformer_y    = transformer_y,
)

forecaster.fit(
    y    = df_test["y"],
)

In [43]:
forecaster.predict(steps = 10)

2024-01-01    116.765062
2024-01-02    123.313042
2024-01-03    126.326493
2024-01-04    128.998787
2024-01-05    132.018021
2024-01-06    134.986518
2024-01-07    137.623172
2024-01-08    139.927627
2024-01-09    141.982424
2024-01-10    143.839301
Freq: D, Name: pred, dtype: float64

In [44]:
# Backtesting predictions
# ==============================================================================
metric, predictions = backtesting_forecaster(
    forecaster            = forecaster,
    y                     = df_test["y"],
    steps                 = 1,
    metric                = 'mean_absolute_error',
    initial_train_size    = len(df_test) - 30,
    interval              = [2.5, 97.5],
    refit                 = True,
    n_jobs                = 1,
    verbose               = True,
    show_progress         = True,
    random_state          = 12345
)


Information of backtesting process
----------------------------------
Number of observations used for initial training: 3958
Number of observations used for backtesting: 30
    Number of folds: 30
    Number skipped folds: 0 
    Number of steps per fold: 1
    Number of steps to exclude from the end of each train set before test (gap): 0

Fold: 0
    Training:   2013-01-30 00:00:00 -- 2023-12-01 00:00:00  (n=3958)
    Validation: 2023-12-02 00:00:00 -- 2023-12-02 00:00:00  (n=1)
Fold: 1
    Training:   2013-01-31 00:00:00 -- 2023-12-02 00:00:00  (n=3958)
    Validation: 2023-12-03 00:00:00 -- 2023-12-03 00:00:00  (n=1)
Fold: 2
    Training:   2013-02-01 00:00:00 -- 2023-12-03 00:00:00  (n=3958)
    Validation: 2023-12-04 00:00:00 -- 2023-12-04 00:00:00  (n=1)
Fold: 3
    Training:   2013-02-02 00:00:00 -- 2023-12-04 00:00:00  (n=3958)
    Validation: 2023-12-05 00:00:00 -- 2023-12-05 00:00:00  (n=1)
Fold: 4
    Training:   2013-02-03 00:00:00 -- 2023-12-05 00:00:00  (n=3958)
    Valid

  0%|          | 0/30 [00:00<?, ?it/s]

In [45]:
transformer_y    = FunctionTransformer(func=np.log1p, inverse_func=np.expm1, validate=True)
transformer_y.fit_transform(df_test[["y"]])

/home/ubuntu/anaconda3/envs/skforecast_13_py12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but FunctionTransformer was fitted with feature names
  warnings.warn(


array([[5.07196368],
       [5.07196368],
       [5.07196368],
       ...,
       [4.77267769],
       [4.55639565],
       [4.6532061 ]])

In [38]:
transformer_y    = StandardScaler()
transformer_y.fit_transform(df_test["y"].values.reshape(-1, 1))

array([[-0.00837553],
       [-0.00837553],
       [-0.00837553],
       ...,
       [-0.16196198],
       [-0.24757884],
       [-0.21153412]])

In [ ]:

forecaster = ForecasterAutoreg(
    regressor        = LinearRegression(),
    lags             = 3, # using past 3 days
    #transformer_y    = transformer_y,  <============
    transformer_exog = transformer_exog
)